In [71]:
import os
import sys
import csv
from enum import Enum

import numpy as np
import pandas as pd

# For Feature Extraction
import tsfresh as tsf
sys.path.append('../../src')
import anoog
import time

In [3]:
df = anoog.io.load_data(data_path='../../data/2021-11-09', 
                        persons=['skadkade', 'vkorzev'], 
                        extraction=anoog.io.extraction_mode.NONE,
                        show_extraction=False,
                        selection=anoog.io.selection_mode.NONE,
                        train_test_split=False) 
df=df.drop(columns = ["y"])

In [4]:
df

,Time,Audio,Voltage,Current,ID
0,2021-11-09 11:51:30.000,-1.484375,20.408691,-0.719727,0.0
1,2021-11-09 11:51:30.010,3.084775,20.402274,-0.807125,0.0
2,2021-11-09 11:51:30.020,-1.039690,20.401973,-0.796764,0.0
3,2021-11-09 11:51:30.030,-1.501513,20.401470,-0.800560,0.0
4,2021-11-09 11:51:30.040,3.164062,20.402350,-0.803417,0.0
...,...,...,...,...,...
24137,2021-11-09 12:24:46.560,-0.077687,19.553144,-0.838514,24.0
24138,2021-11-09 12:24:46.570,0.538470,19.554451,-0.827846,24.0
24139,2021-11-09 12:24:46.580,-0.509043,19.554166,-0.829795,24.0
24140,2021-11-09 12:24:46.590,0.256057,19.555244,-0.832509,24.0


In [69]:
#Funktioniert nicht, da tsfresh eine komplexe art der Variablenbezeichnung hat
features = ["Audio__length",
        "Audio__count_below_mean",
        "Audio__cid_ce_normalize_true",
        "Audio__number_peaks"]

def magic(features):
    tup = dict()
    for i in features:
        if "Audio__" in i:
            tup["Audio"] = dict()
        if "Current__" in i:
             tup["Current"] = dict()
        if "Voltage__" in i:
             tup["Voltage"] = dict()

    for i in features:
        if "Audio__" in i[0:7]:
            tup["Audio"][i[7:]] = []
        if "Current__"[::-1] in i[8::-1]:
            tup["Current"][i[8:]] = []
        if "Voltage__"[::-1] in i[8::-1]:
            tup["Voltage"][i[8:]] = []
            
    return tup

print(magic(features))

{'Audio': {'length': [], 'count_below_mean': [], 'cid_ce_normalize_true': [], 'number_peaks': []}}


In [89]:
# Variance Thresholding
start = time.time()
tsf.feature_extraction.extraction.extract_features(df,column_id="ID",kind_to_fc_parameters={'Audio': {"variation_coefficient": [], "fft_aggregated":[{"aggtype": "variance"}]}, "Voltage": {"sum_values": [], "abs_energy": [], "sum_of_reoccurring_data_points": [], "fft_coefficient": [{"attr": "real","coeff":0}], "fft_coefficient": [{"attr":"abs","coeff":0}], "fft_aggregated": [{"aggtype":"variance"}], "friedrich_coefficients": [{"coeff":3,"m":3,"r":30}]}, "Current": {"abs_energy": [], "fft_aggregated":[{"aggtype":"variance"}]}})
ende = time.time()
print(ende-start)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.34s/it]


26.931265354156494


In [91]:
# Modelbased Selection
start = time.time()
tsf.feature_extraction.extraction.extract_features(df,column_id="ID",kind_to_fc_parameters={'Audio': {"count_above_mean":[]}})
ende = time.time()
print(ende-start)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.33s/it]

26.643812656402588


- SelektKBest: 28.98436403274536 
- Variance Thresholding: 26.931265354156494 - 31.487123250961304
- Modelbased Selection: 26.37845492362976 - 31.235893487930298

### SelektKBest: 
- Audio__length
- Audio__count_below_mean
- Audio__cid_ce__normalize_True
- Audio__number_peaks__n_1
- Audio__number_peaks__n_5

### Variance Thresholding: 
- Audio__variation_coefficient
- Audio__fft_aggregated__aggtype_"variance"
- Voltage__sum_values
- Voltage__abs_energy Voltage__sum_of_reoccurring_data_points
- Voltage__fft_coefficient__attr_"real"__coeff_0
- Voltage__fft_coefficient__attr_"abs"__coeff_0
- Voltage__fft_aggregated__aggtype_"variance"
- Voltage__friedrich_coefficients__coeff_3__m_3__r_30
- Current__abs_energy
- Current__fft_aggregated__aggtype_"variance"

### Modelbased Selection
- Audio__count_above_mean